In [1]:
import py21cmfast as p21c
#print("py21cmfast version:", p21c.__version__)
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.special import wofz

/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


In [2]:
if not os.path.exists("_cache"):
    os.mkdir("_cache")
p21c.config["direc"] = "_cache"
p21c.cache_tools.clear_cache(direc="_cache")


def build_21cm_fields(HII_DIM, BOX_LEN, redshift, RANDOM_SEED):
    """
    21cmfast-only part of your script:
      - Sets up cache directory (but DOES NOT clear it)
      - Runs coeval + perturb_field (+ ionize_box, as in your script)
      - Builds:
          density_field  = perturbed_field.density                  (delta = rho/rho_mean - 1)
          velocity_field = coeval.lowres_vx                          (LOS vx, km/s)
          temp_field     = T0 * ( (1 + density_field) ** (gamma-1) ) (your formula)

    Assumes imports are done outside:
      import os
      import py21cmfast as p21c
      import numpy as np
    """

    # -------------------------
    # 0) py21cmfast cache setup (same as your script, BUT no clear_cache here)
    # -------------------------
    if not os.path.exists("_cache"):
        os.mkdir("_cache")
    p21c.config["direc"] = "_cache"

    # -------------------------
    # 1) Fixed box/setup params (exactly as your script)
    # -------------------------
    seed = RANDOM_SEED

    user_params = {
        "HII_DIM": int(HII_DIM),
        "BOX_LEN": float(BOX_LEN),
        "USE_INTERPOLATION_TABLES": True,
    }
    cosmo_params = p21c.CosmoParams(SIGMA_8=0.8)
    astro_params = p21c.AstroParams({"HII_EFF_FACTOR": 20.0})

    # initial_conditions created in your script (kept here)
    _ = p21c.initial_conditions(
        user_params=user_params,
        cosmo_params=cosmo_params,
        random_seed=seed,
    )

    z_bar = float(redshift)

    # -------------------------
    # 2) Generate fields at this redshift
    # -------------------------
    coeval = p21c.run_coeval(
        redshift=z_bar,
        user_params=user_params,
        cosmo_params=cosmo_params,
        astro_params=astro_params,
        random_seed=seed,
    )

    perturbed_field = p21c.perturb_field(
        redshift=z_bar,
        user_params=user_params,
        cosmo_params=cosmo_params,
        random_seed=seed,
    )
    # kept exactly as your script (even though not used later)
    #_ = p21c.ionize_box(perturbed_field=perturbed_field)

    # -------------------------
    # 3) Build the three output fields (same logic as your 1D extraction)
    # -------------------------
    density_field = perturbed_field.density  # delta = rho/rho_mean - 1
     # Delta = rho/rho_mean

    velocity_field_kms = coeval.lowres_vx    # LOS velocity along x (km/s)

    return velocity_field_kms, density_field


In [3]:
def lyalpha_forest_from_fields(velocity_field_kms, density_field, y0, z0, redshift, BOX_LEN, T0, gamma):
    """
    Everything-after-21cmfast part of your script:
      - Extracts 1D LOS along x at fixed (y0, z0)
      - Reuses your mapping z(x), wavelength grid, nHI, tau integral, flux, sorting
      - Returns the same dictionary outputs as your original function
        (except it does NOT return coeval/perturbed_field/ionized_field objects,
         since inputs are only the three fields)

    Assumes imports are done outside:
      import numpy as np
      from scipy.special import wofz
    """
    overdensity_field = 1.0 + density_field 
    temp_field_K = T0 * (overdensity_field ** (gamma - 1.0))
    # -------------------------
    # 3) Extract 1D sightline (LOS = x-axis) (same as your script)
    # -------------------------
    density_1d = density_field[:, y0, z0]         # delta = rho/rho_mean - 1
    overdensity_1d = 1.0 + density_1d             # Delta = rho/rho_mean

    vlos_1d_kms = velocity_field_kms[:, y0, z0]   # LOS velocity along x (km/s)

    # temperature on the LOS from the provided temperature field
    T_1d_K = temp_field_K[:, y0, z0]

    # -------------------------
    # 4) Build per-pixel redshift z(x) and wavelength grid (your mapping logic)
    # -------------------------
    z_bar = float(redshift)

    N = int(density_field.shape[0])
    L_box_hinv_Mpc = float(BOX_LEN)

    h = 0.7
    Omega_m = 0.3
    Omega_L = 0.7

    c_kms = 299792.458
    lambda_alpha_A = 1215.67

    dchi_hinv_Mpc = L_box_hinv_Mpc / N
    chi_hinv_Mpc = (np.arange(N) - (N - 1) / 2.0) * dchi_hinv_Mpc
    chi_Mpc = chi_hinv_Mpc / h

    H0 = 100.0 * h  # km/s/Mpc

    def Eofz(z):
        return np.sqrt(Omega_m * (1.0 + z) ** 3 + Omega_L)

    def Hofz(z):
        return H0 * Eofz(z)

    z_pix = np.empty(N, dtype=np.float64)
    ic = (N - 1) // 2
    z_pix[ic] = z_bar

    for i in range(ic + 1, N):
        dchi = chi_Mpc[i] - chi_Mpc[i - 1]
        z_pix[i] = z_pix[i - 1] + (Hofz(z_pix[i - 1]) / c_kms) * dchi

    for i in range(ic - 1, -1, -1):
        dchi = chi_Mpc[i + 1] - chi_Mpc[i]
        z_pix[i] = z_pix[i + 1] - (Hofz(z_pix[i + 1]) / c_kms) * dchi

    lambda_center_A = lambda_alpha_A * (1.0 + z_pix)

    # -------------------------
    # 5) nHI from photoionization equilibrium (your exact blocks)
    # -------------------------
    # Constants (cgs)
    kB = 1.380649e-16
    m_p = 1.67262192369e-24
    G_cgs = 6.67430e-8

    Omega_b = 0.048
    Y_He = 0.24

    # ---- UV background Γ_HI(z): use nearest tabulated value to the input redshift z_bar ----
    Gamma_HI_table = {
        1.49: 0.732e-12,
        1.61: 0.799e-12,
        1.74: 0.859e-12,
        1.87: 0.909e-12,
        2.01: 0.944e-12,
        2.16: 0.963e-12,
        2.32: 0.965e-12,
        2.48: 0.950e-12,
        2.65: 0.919e-12,
        2.83: 0.875e-12,
        3.02: 0.822e-12,
        3.21: 0.765e-12,
        3.42: 0.705e-12,
        3.64: 0.647e-12,
        3.87: 0.594e-12,
        4.11: 0.546e-12,
        4.36: 0.504e-12,
        4.62: 0.469e-12,
        4.89: 0.441e-12,
        5.18: 0.412e-12,
        5.49: 0.360e-12,
        5.81: 0.293e-12,
        6.14: 0.230e-12,
        6.49: 0.175e-12,
        6.86: 0.129e-12,
        7.25: 0.928e-13,
    }

    z_tab = np.array(list(Gamma_HI_table.keys()), dtype=np.float64)
    G_tab = np.array(list(Gamma_HI_table.values()), dtype=np.float64)
    Gamma_HI = G_tab[np.argmin(np.abs(z_tab - z_bar))]  # s^-1

    # Doppler parameter
    b_cms = np.sqrt(2.0 * kB * T_1d_K / m_p)

    # Mean hydrogen density nHbar(z)
    km_s_Mpc_to_sinv = 1e5 / 3.0856775814913673e24
    H0_sinv = (100.0 * h) * km_s_Mpc_to_sinv

    rho_crit0 = 3.0 * H0_sinv**2 / (8.0 * np.pi * G_cgs)
    rho_b0 = Omega_b * rho_crit0

    X_H = 1.0 - Y_He
    nHbar = (X_H * rho_b0 / m_p) * (1.0 + z_pix) ** 3
    nH = overdensity_1d * nHbar

    nHe_over_nH = Y_He / (4.0 * X_H)
    ne = nH * (1.0 + 2.0 * nHe_over_nH)

    # Recomb coeff (simple fit)
    alphaA = 4.2e-13 * (T_1d_K / 1e4) ** (-0.7)

    # Ferland et al. style fit (your block)
    kB_eV_per_K = 8.617333262145e-5
    T_eV = T_1d_K * kB_eV_per_K
    lnT = np.log(T_eV)

    alphaA2 = np.exp(
        -28.6130338
        - 0.72411256 * lnT
        - 2.02604473e-2 * lnT**2
        - 2.38086188e-3 * lnT**3
        - 3.21260521e-4 * lnT**4
        - 1.42150291e-5 * lnT**5
        + 4.98910892e-6 * lnT**6
        + 5.75561414e-7 * lnT**7
        - 1.85676704e-8 * lnT**8
        - 3.07113524e-9 * lnT**9
    )

    nHI = alphaA * ne * nH / Gamma_HI
    nHI2 = alphaA2 * ne * nH / Gamma_HI

    # -------------------------
    # 6) Optical depth integral with Voigt profile (your exact structure)
    # -------------------------
    c_cms = 2.99792458e10
    lambda_alpha_cm = 1215.67e-8
    I_alpha = 4.45e-18
    Gamma_Lya = 6.265e8

    Mpc_cm = 3.0856775814913673e24
    dx_com_cm = (L_box_hinv_Mpc / h) * Mpc_cm / N

    z0_eval = z_pix.copy()
    lambda0_A = 1215.67 * (1.0 + z0_eval)

    def voigt_H(a, u):
        return np.real(wofz(u + 1j * a))

    v_cms = vlos_1d_kms * 1e5
    a_i = (Gamma_Lya * lambda_alpha_cm) / (4.0 * np.pi * b_cms)

    pref_i = nHI / (b_cms * (1.0 + z_pix))
    pref_i2 = nHI2 / (b_cms * (1.0 + z_pix))

    global_pref = (c_cms * I_alpha / np.sqrt(np.pi)) * dx_com_cm

    tau = np.empty_like(z0_eval)
    tau2 = np.empty_like(z0_eval)

    for j in range(N):
        delta_v_cosmo = c_cms * (z_pix - z0_eval[j]) / (1.0 + z0_eval[j])
        u_ij = (delta_v_cosmo + v_cms) / b_cms
        tau[j] = global_pref * np.sum(pref_i * voigt_H(a_i, u_ij))
        tau2[j] = global_pref * np.sum(pref_i2 * voigt_H(a_i, u_ij))

    F = np.exp(-tau)
    F2 = np.exp(-tau2)

    # Sort by wavelength (as you do before plotting)
    order = np.argsort(lambda0_A)

    return {
        "redshift_center": z_bar,
        "y0": y0,
        "z0": z0,
        "lambda_A": lambda0_A[order],
        "flux_F": F[order],
        "flux_F2": F2[order],
        "tau": tau[order],
        "tau2": tau2[order],
        "z_pix": z0_eval[order],
        "lambda_center_A": lambda_center_A[order],
        "overdensity_1d": overdensity_1d.astype(np.float32),
        "vlos_1d_kms": vlos_1d_kms.astype(np.float32),
        "T_1d_K": T_1d_K.astype(np.float32),
    }


In [4]:
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Grid controls
N_T0    = 5
N_GAMMA = 5
N_SIGHTLINES_PER_GRID = 100

T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# ---- Noise controls (Option C) ----
SNR_FIXED = 20.0        # pick one value
EPS_FLUX  = 1e-6        # avoid log(0) when flux + noise <= 0

RNG_SEED = 123

# --- NEW: number of box realizations ---
N_REALIZATIONS = 40

# Total simulations now multiplies by number of realizations
N_SIMS = N_REALIZATIONS * N_T0 * N_GAMMA * N_SIGHTLINES_PER_GRID

OUTFILE = (
    f"lyalpha_train_z{REDSHIFT:.2f}_grid{N_T0}x{N_GAMMA}_"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNRlogU{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build 21cm fields ONCE
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) Build the 5x5 (T0, gamma) grid
# ----------------------------
logT0_vals = np.linspace(np.log(T0_MIN), np.log(T0_MAX), N_T0, dtype=np.float64)
T0_grid = np.exp(logT0_vals).astype(np.float32)  # (5,)
gamma_grid = np.linspace(GAMMA_MIN, GAMMA_MAX, N_GAMMA).astype(np.float32)  # (5,)

# ----------------------------
# 2) RNG + one dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

y0_test = int(rng.integers(0, HII_DIM, dtype=np.int32))
z0_test = int(rng.integers(0, HII_DIM, dtype=np.int32))

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_grid[0]),
    gamma=float(gamma_grid[0]),
    y0=y0_test,
    z0=z0_test
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X = np.empty((N_SIMS, Npix), dtype=np.float32)
theta = np.empty((N_SIMS, 2), dtype=np.float32)   # [:,0]=T0, [:,1]=gamma
yz = np.empty((N_SIMS, 2), dtype=np.int32)        # [:,0]=y0, [:,1]=z0
## ----------------------------
# 3) Generate 100 random sightlines for each (T0, gamma) grid point
# ----------------------------
idx = 0

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    # --- your existing loops go here (unchanged, just indented) ---
    for T0 in T0_grid:
        for gamma in gamma_grid:
            # 100 random (y0, z0) for THIS grid point (and this realization)
            y0_samples = rng.integers(0, HII_DIM, size=N_SIGHTLINES_PER_GRID, dtype=np.int32)
            z0_samples = rng.integers(0, HII_DIM, size=N_SIGHTLINES_PER_GRID, dtype=np.int32)

            for k in range(N_SIGHTLINES_PER_GRID):
                y0 = int(y0_samples[k])
                z0 = int(z0_samples[k])

                out = lyalpha_forest_from_fields(
                    velocity_field_kms=velocity_field_kms,
                    density_field=density_field,
                    redshift=REDSHIFT,
                    BOX_LEN=BOX_LEN,
                    T0=float(T0),
                    gamma=float(gamma),
                    y0=y0,
                    z0=z0
                )

                F = np.asarray(out["flux_F2"], dtype=np.float32)

                # Fixed-SNR Gaussian noise in FLUX space
                sigma = 1.0 / float(SNR_FIXED)
                noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)

                F_noisy = np.clip(F + noise, 0.0, 1.0).astype(np.float32)
                X[idx, :] = F_noisy
                #snr[idx] = snr_i
                theta[idx, 0] = T0
                theta[idx, 1] = gamma
                yz[idx, 0] = y0
                yz[idx, 1] = z0

                idx += 1
                if (idx % 10 == 0) or (idx == N_SIMS):
                    print(f"[{idx}/{N_SIMS}] done")


# ----------------------------
# 4) Save dataset (same as before, plus optional snr array)
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "T0_grid_K": T0_grid.astype(np.float32),
    "gamma_grid": gamma_grid.astype(np.float32),
    "N_T0": int(N_T0),
    "N_gamma": int(N_GAMMA),
    "N_sightlines_per_grid": int(N_SIGHTLINES_PER_GRID),

    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": "[0.0, 1.0]", 
    "X_definition": "X is noisy transmitted flux F_noisy",
    "summary": "X is noisy flux_F2 arrays; theta[:,0]=T0[K], theta[:,1]=gamma; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    #snr=snr,  # optional but recommended
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)


/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarni

[10/100000] done
[20/100000] done
[30/100000] done
[40/100000] done
[50/100000] done
[60/100000] done
[70/100000] done
[80/100000] done
[90/100000] done
[100/100000] done
[110/100000] done
[120/100000] done
[130/100000] done
[140/100000] done
[150/100000] done
[160/100000] done
[170/100000] done
[180/100000] done
[190/100000] done
[200/100000] done
[210/100000] done
[220/100000] done
[230/100000] done
[240/100000] done
[250/100000] done
[260/100000] done
[270/100000] done
[280/100000] done
[290/100000] done
[300/100000] done
[310/100000] done
[320/100000] done
[330/100000] done
[340/100000] done
[350/100000] done
[360/100000] done
[370/100000] done
[380/100000] done
[390/100000] done
[400/100000] done
[410/100000] done
[420/100000] done
[430/100000] done
[440/100000] done
[450/100000] done
[460/100000] done
[470/100000] done
[480/100000] done
[490/100000] done
[500/100000] done
[510/100000] done
[520/100000] done
[530/100000] done
[540/100000] done
[550/100000] done
[560/100000] done
[

In [5]:
p21c.cache_tools.clear_cache(direc="_cache")

In [6]:
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Grid controls
N_T0    = 5
N_GAMMA = 5
N_SIGHTLINES_PER_GRID = 100

T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# ---- Noise controls (Option C) ----
SNR_FIXED = 20.0        # pick one value
EPS_FLUX  = 1e-6        # avoid log(0) when flux + noise <= 0

RNG_SEED = 321

# --- NEW: number of box realizations ---
N_REALIZATIONS = 10

# Total simulations now multiplies by number of realizations
N_SIMS = N_REALIZATIONS * N_T0 * N_GAMMA * N_SIGHTLINES_PER_GRID

OUTFILE = (
    f"lyalpha_test_z{REDSHIFT:.2f}_grid{N_T0}x{N_GAMMA}_"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNRlogU{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build 21cm fields ONCE
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) Build the 5x5 (T0, gamma) grid
# ----------------------------
logT0_vals = np.linspace(np.log(T0_MIN), np.log(T0_MAX), N_T0, dtype=np.float64)
T0_grid = np.exp(logT0_vals).astype(np.float32)  # (5,)
gamma_grid = np.linspace(GAMMA_MIN, GAMMA_MAX, N_GAMMA).astype(np.float32)  # (5,)

# ----------------------------
# 2) RNG + one dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

y0_test = int(rng.integers(0, HII_DIM, dtype=np.int32))
z0_test = int(rng.integers(0, HII_DIM, dtype=np.int32))

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_grid[0]),
    gamma=float(gamma_grid[0]),
    y0=y0_test,
    z0=z0_test
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X = np.empty((N_SIMS, Npix), dtype=np.float32)
theta = np.empty((N_SIMS, 2), dtype=np.float32)   # [:,0]=T0, [:,1]=gamma
yz = np.empty((N_SIMS, 2), dtype=np.int32)        # [:,0]=y0, [:,1]=z0
## ----------------------------
# 3) Generate 100 random sightlines for each (T0, gamma) grid point
# ----------------------------
idx = 0

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    # --- your existing loops go here (unchanged, just indented) ---
    for T0 in T0_grid:
        for gamma in gamma_grid:
            # 100 random (y0, z0) for THIS grid point (and this realization)
            y0_samples = rng.integers(0, HII_DIM, size=N_SIGHTLINES_PER_GRID, dtype=np.int32)
            z0_samples = rng.integers(0, HII_DIM, size=N_SIGHTLINES_PER_GRID, dtype=np.int32)

            for k in range(N_SIGHTLINES_PER_GRID):
                y0 = int(y0_samples[k])
                z0 = int(z0_samples[k])

                out = lyalpha_forest_from_fields(
                    velocity_field_kms=velocity_field_kms,
                    density_field=density_field,
                    redshift=REDSHIFT,
                    BOX_LEN=BOX_LEN,
                    T0=float(T0),
                    gamma=float(gamma),
                    y0=y0,
                    z0=z0
                )

                F = np.asarray(out["flux_F2"], dtype=np.float32)

                # Fixed-SNR Gaussian noise in FLUX space
                sigma = 1.0 / float(SNR_FIXED)
                noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)

                F_noisy = np.clip(F + noise, 0.0, 1.0).astype(np.float32)
                X[idx, :] = F_noisy
                #snr[idx] = snr_i
                theta[idx, 0] = T0
                theta[idx, 1] = gamma
                yz[idx, 0] = y0
                yz[idx, 1] = z0

                idx += 1
                if (idx % 10 == 0) or (idx == N_SIMS):
                    print(f"[{idx}/{N_SIMS}] done")


# ----------------------------
# 4) Save dataset (same as before, plus optional snr array)
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "T0_grid_K": T0_grid.astype(np.float32),
    "gamma_grid": gamma_grid.astype(np.float32),
    "N_T0": int(N_T0),
    "N_gamma": int(N_GAMMA),
    "N_sightlines_per_grid": int(N_SIGHTLINES_PER_GRID),

    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": "[0.0, 1.0]", 
    "X_definition": "X is noisy transmitted flux F_noisy",
    "summary": "X is noisy flux_F2 arrays; theta[:,0]=T0[K], theta[:,1]=gamma; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    #snr=snr,  # optional but recommended
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)


[10/25000] done
[20/25000] done
[30/25000] done
[40/25000] done
[50/25000] done
[60/25000] done
[70/25000] done
[80/25000] done
[90/25000] done
[100/25000] done
[110/25000] done
[120/25000] done
[130/25000] done
[140/25000] done
[150/25000] done
[160/25000] done
[170/25000] done
[180/25000] done
[190/25000] done
[200/25000] done
[210/25000] done
[220/25000] done
[230/25000] done
[240/25000] done
[250/25000] done
[260/25000] done
[270/25000] done
[280/25000] done
[290/25000] done
[300/25000] done
[310/25000] done
[320/25000] done
[330/25000] done
[340/25000] done
[350/25000] done
[360/25000] done
[370/25000] done
[380/25000] done
[390/25000] done
[400/25000] done
[410/25000] done
[420/25000] done
[430/25000] done
[440/25000] done
[450/25000] done
[460/25000] done
[470/25000] done
[480/25000] done
[490/25000] done
[500/25000] done
[510/25000] done
[520/25000] done
[530/25000] done
[540/25000] done
[550/25000] done
[560/25000] done
[570/25000] done
[580/25000] done
[590/25000] done
[600/2

In [11]:
p21c.cache_tools.clear_cache(direc="_cache")

In [4]:
#alternatively using uniform distribution over T0 and gamma instead of grid
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift, RANDOM_SEED) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict

import numpy as np

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Uniform parameter ranges (replaces the grid)
T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# Noise controls (fixed SNR in FLUX space)
SNR_FIXED = 20.0
EPS_FLUX  = 1e-6   # to avoid exact 0/1 after clipping (useful for later transforms)

RNG_SEED = 123

# Box realizations + sims per realization
N_REALIZATIONS = 160
N_SIMS_PER_REALIZATION = 625 # e.g. match old 5*5*100 = 2500
N_SIMS = N_REALIZATIONS * N_SIMS_PER_REALIZATION

OUTFILE = (
    f"lyalpha_train_z{REDSHIFT:.2f}_uniformT0gamma_"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNR{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build 21cm fields ONCE (only for a dry run to get Npix)
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) RNG + one deterministic dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

T0_probe    = 0.5 * (T0_MIN + T0_MAX)
gamma_probe = 0.5 * (GAMMA_MIN + GAMMA_MAX)

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_probe),
    gamma=float(gamma_probe),
    y0=0,
    z0=0
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X     = np.empty((N_SIMS, Npix), dtype=np.float32)     # noisy transmitted flux
theta = np.empty((N_SIMS, 2), dtype=np.float32)        # [:,0]=T0, [:,1]=gamma
yz    = np.empty((N_SIMS, 2), dtype=np.int32)          # [:,0]=y0, [:,1]=z0

# ----------------------------
# 2) Generate dataset: uniform (T0, gamma), random (y0, z0), per-realization fields
# ----------------------------
idx = 0
sigma = 1.0 / float(SNR_FIXED)

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    for n in range(N_SIMS_PER_REALIZATION):

        # Uniform draws of parameters
        T0    = float(rng.uniform(T0_MIN, T0_MAX))
        gamma = float(rng.uniform(GAMMA_MIN, GAMMA_MAX))

        # Random sightline position (y0, z0)
        y0 = int(rng.integers(0, HII_DIM, dtype=np.int32))
        z0 = int(rng.integers(0, HII_DIM, dtype=np.int32))

        out = lyalpha_forest_from_fields(
            velocity_field_kms=velocity_field_kms,
            density_field=density_field,
            redshift=REDSHIFT,
            BOX_LEN=BOX_LEN,
            T0=T0,
            gamma=gamma,
            y0=y0,
            z0=z0
        )

        F = np.asarray(out["flux_F2"], dtype=np.float32)

        # Fixed-SNR Gaussian noise in FLUX space
        noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)
        F_noisy = np.clip(F + noise, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)

        # Store
        X[idx, :] = F_noisy
        theta[idx, 0] = T0
        theta[idx, 1] = gamma
        yz[idx, 0] = y0
        yz[idx, 1] = z0

        idx += 1
        if (idx % 1000 == 0) or (idx == N_SIMS):
            print(f"[{idx}/{N_SIMS}] done")

assert idx == N_SIMS, f"Filled idx={idx} rows, expected N_SIMS={N_SIMS}"

# ----------------------------
# 3) Save dataset
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "theta_sampling": "uniform_linear",
    "T0_range_K": [float(T0_MIN), float(T0_MAX)],
    "gamma_range": [float(GAMMA_MIN), float(GAMMA_MAX)],
    "N_realizations": int(N_REALIZATIONS),
    "N_sims_per_realization": int(N_SIMS_PER_REALIZATION),

    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": f"[{EPS_FLUX}, {1.0 - EPS_FLUX}]",
    "X_definition": "X is noisy transmitted flux F_noisy",
    "summary": "X is noisy flux_F2 arrays; theta[:,0]=T0[K], theta[:,1]=gamma; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)


/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarni

[1000/100000] done
[2000/100000] done
[3000/100000] done
[4000/100000] done
[5000/100000] done
[6000/100000] done
[7000/100000] done
[8000/100000] done
[9000/100000] done
[10000/100000] done
[11000/100000] done
[12000/100000] done
[13000/100000] done
[14000/100000] done
[15000/100000] done
[16000/100000] done
[17000/100000] done
[18000/100000] done
[19000/100000] done
[20000/100000] done
[21000/100000] done
[22000/100000] done
[23000/100000] done
[24000/100000] done
[25000/100000] done
[26000/100000] done
[27000/100000] done
[28000/100000] done
[29000/100000] done
[30000/100000] done
[31000/100000] done
[32000/100000] done
[33000/100000] done
[34000/100000] done
[35000/100000] done
[36000/100000] done
[37000/100000] done
[38000/100000] done
[39000/100000] done
[40000/100000] done
[41000/100000] done
[42000/100000] done
[43000/100000] done
[44000/100000] done
[45000/100000] done
[46000/100000] done
[47000/100000] done
[48000/100000] done
[49000/100000] done
[50000/100000] done
[51000/10

In [5]:
p21c.cache_tools.clear_cache(direc="_cache")

In [6]:
#alternatively using uniform distribution over T0 and gamma instead of grid
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift, RANDOM_SEED) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict

import numpy as np

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Uniform parameter ranges (replaces the grid)
T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# Noise controls (fixed SNR in FLUX space)
SNR_FIXED = 20.0
EPS_FLUX  = 1e-6   # to avoid exact 0/1 after clipping (useful for later transforms)

RNG_SEED = 321

# Box realizations + sims per realization
N_REALIZATIONS = 40
N_SIMS_PER_REALIZATION = 625  # e.g. match old 5*5*100 = 2500
N_SIMS = N_REALIZATIONS * N_SIMS_PER_REALIZATION

OUTFILE = (
    f"lyalpha_test_z{REDSHIFT:.2f}_uniformT0gamma_"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNR{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build 21cm fields ONCE (only for a dry run to get Npix)
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) RNG + one deterministic dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

T0_probe    = 0.5 * (T0_MIN + T0_MAX)
gamma_probe = 0.5 * (GAMMA_MIN + GAMMA_MAX)

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_probe),
    gamma=float(gamma_probe),
    y0=0,
    z0=0
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X     = np.empty((N_SIMS, Npix), dtype=np.float32)     # noisy transmitted flux
theta = np.empty((N_SIMS, 2), dtype=np.float32)        # [:,0]=T0, [:,1]=gamma
yz    = np.empty((N_SIMS, 2), dtype=np.int32)          # [:,0]=y0, [:,1]=z0

# ----------------------------
# 2) Generate dataset: uniform (T0, gamma), random (y0, z0), per-realization fields
# ----------------------------
idx = 0
sigma = 1.0 / float(SNR_FIXED)

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    for n in range(N_SIMS_PER_REALIZATION):

        # Uniform draws of parameters
        T0    = float(rng.uniform(T0_MIN, T0_MAX))
        gamma = float(rng.uniform(GAMMA_MIN, GAMMA_MAX))

        # Random sightline position (y0, z0)
        y0 = int(rng.integers(0, HII_DIM, dtype=np.int32))
        z0 = int(rng.integers(0, HII_DIM, dtype=np.int32))

        out = lyalpha_forest_from_fields(
            velocity_field_kms=velocity_field_kms,
            density_field=density_field,
            redshift=REDSHIFT,
            BOX_LEN=BOX_LEN,
            T0=T0,
            gamma=gamma,
            y0=y0,
            z0=z0
        )

        F = np.asarray(out["flux_F2"], dtype=np.float32)

        # Fixed-SNR Gaussian noise in FLUX space
        noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)
        F_noisy = np.clip(F + noise, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)

        # Store
        X[idx, :] = F_noisy
        theta[idx, 0] = T0
        theta[idx, 1] = gamma
        yz[idx, 0] = y0
        yz[idx, 1] = z0

        idx += 1
        if (idx % 1000 == 0) or (idx == N_SIMS):
            print(f"[{idx}/{N_SIMS}] done")

assert idx == N_SIMS, f"Filled idx={idx} rows, expected N_SIMS={N_SIMS}"

# ----------------------------
# 3) Save dataset
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "theta_sampling": "uniform_linear",
    "T0_range_K": [float(T0_MIN), float(T0_MAX)],
    "gamma_range": [float(GAMMA_MIN), float(GAMMA_MAX)],
    "N_realizations": int(N_REALIZATIONS),
    "N_sims_per_realization": int(N_SIMS_PER_REALIZATION),

    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": f"[{EPS_FLUX}, {1.0 - EPS_FLUX}]",
    "X_definition": "X is noisy transmitted flux F_noisy",
    "summary": "X is noisy flux_F2 arrays; theta[:,0]=T0[K], theta[:,1]=gamma; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)


[1000/25000] done
[2000/25000] done
[3000/25000] done
[4000/25000] done
[5000/25000] done
[6000/25000] done
[7000/25000] done
[8000/25000] done
[9000/25000] done
[10000/25000] done
[11000/25000] done
[12000/25000] done
[13000/25000] done
[14000/25000] done
[15000/25000] done
[16000/25000] done
[17000/25000] done
[18000/25000] done
[19000/25000] done
[20000/25000] done
[21000/25000] done
[22000/25000] done
[23000/25000] done
[24000/25000] done
[25000/25000] done
Saved: lyalpha_test_z2.50_uniformT0gamma_Nsims25000_R40_SNR20.npz
X shape: (25000, 100) theta shape: (25000, 2) yz shape: (25000, 2)


In [4]:
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift, RANDOM_SEED) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict with key "flux_F2"

import numpy as np

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Uniform parameter ranges (T0, gamma)
T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# Nuisance parameter: tau scaling A = exp(logA)
# tau' = A * tau_true  ->  F' = exp(-A*tau_true) = F_true**A
LOGA_MIN, LOGA_MAX = -0.4, 0.4   # A in ~[0.67, 1.49]

# Noise controls (fixed SNR in FLUX space)
SNR_FIXED = 20.0
EPS_FLUX  = 1e-6  # keep flux away from exactly 0/1 (stable logs / transforms)
RNG_SEED  = 123

# Box realizations + sims per realization
N_REALIZATIONS = 80
N_SIMS_PER_REALIZATION = 1250
N_SIMS = N_REALIZATIONS * N_SIMS_PER_REALIZATION

OUTFILE = (
    f"lyalpha_train_z{REDSHIFT:.2f}_uniformT0gamma_logAUniform"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNR{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build fields ONCE for a dry run to get Npix
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) RNG + deterministic dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

T0_probe    = 0.5 * (T0_MIN + T0_MAX)
gamma_probe = 0.5 * (GAMMA_MIN + GAMMA_MAX)

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_probe),
    gamma=float(gamma_probe),
    y0=0,
    z0=0
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X     = np.empty((N_SIMS, Npix), dtype=np.float32)     # noisy transmitted flux after nuisance tau scaling
theta = np.empty((N_SIMS, 3), dtype=np.float32)        # [:,0]=T0, [:,1]=gamma, [:,2]=logA
yz    = np.empty((N_SIMS, 2), dtype=np.int32)          # [:,0]=y0, [:,1]=z0

# Precompute flux-noise sigma for fixed SNR
sigma = 1.0 / float(SNR_FIXED)

# ----------------------------
# 2) Generate dataset
# ----------------------------
idx = 0

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    for n in range(N_SIMS_PER_REALIZATION):

        # --- Uniform draws of physical parameters ---
        T0    = float(rng.uniform(T0_MIN, T0_MAX))
        gamma = float(rng.uniform(GAMMA_MIN, GAMMA_MAX))

        # --- Uniform draw of nuisance parameter logA (A = exp(logA)) ---
        logA = float(rng.uniform(LOGA_MIN, LOGA_MAX))
        A = float(np.exp(logA))

        # --- Random sightline position ---
        y0 = int(rng.integers(0, HII_DIM, dtype=np.int32))
        z0 = int(rng.integers(0, HII_DIM, dtype=np.int32))

        out = lyalpha_forest_from_fields(
            velocity_field_kms=velocity_field_kms,
            density_field=density_field,
            redshift=REDSHIFT,
            BOX_LEN=BOX_LEN,
            T0=T0,
            gamma=gamma,
            y0=y0,
            z0=z0
        )

        # Noiseless transmitted flux from your forward model
        F = np.asarray(out["flux_F2"], dtype=np.float32)

        # --- Apply nuisance in tau-space: tau' = A * tau_true ---
        # Safety clipping before log
        F = np.clip(F, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)
        tau = (-np.log(F)).astype(np.float32)
        F_scaled = np.exp(-A * tau).astype(np.float32)

        # --- Add Gaussian noise in FLUX space ---
        noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)
        F_noisy = np.clip(F_scaled + noise, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)

        # Store
        X[idx, :] = F_noisy
        theta[idx, 0] = T0
        theta[idx, 1] = gamma
        theta[idx, 2] = logA
        yz[idx, 0] = y0
        yz[idx, 1] = z0

        idx += 1
        if (idx % 1000 == 0) or (idx == N_SIMS):
            print(f"[{idx}/{N_SIMS}] done")

assert idx == N_SIMS, f"Filled idx={idx} rows, expected N_SIMS={N_SIMS}"

# ----------------------------
# 3) Save dataset
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "theta_sampling": "uniform_linear",
    "T0_range_K": [float(T0_MIN), float(T0_MAX)],
    "gamma_range": [float(GAMMA_MIN), float(GAMMA_MAX)],
    "logA_range": [float(LOGA_MIN), float(LOGA_MAX)],
    "theta_columns": ["T0_K", "gamma", "logA"],

    "nuisance_definition": "A = exp(logA); tau_scaled = A * tau_true; F_scaled = exp(-tau_scaled) = F_true**A",

    "N_realizations": int(N_REALIZATIONS),
    "N_sims_per_realization": int(N_SIMS_PER_REALIZATION),
    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian_after_tau_scaling",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": f"[{EPS_FLUX}, {1.0 - EPS_FLUX}]",
    "X_definition": "X is noisy transmitted flux after tau-scaling nuisance: F_noisy = clip(exp(-exp(logA)*(-ln(F_true))) + noise)",
    "summary": "X is noisy flux after tau-scaling; theta[:,0]=T0[K], theta[:,1]=gamma, theta[:,2]=logA; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)
print("theta columns:", meta["theta_columns"])
print("theta ranges:",
      f"T0[{theta[:,0].min():.1f}, {theta[:,0].max():.1f}]",
      f"gamma[{theta[:,1].min():.3f}, {theta[:,1].max():.3f}]",
      f"logA[{theta[:,2].min():.3f}, {theta[:,2].max():.3f}]")


/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/home/akshat/miniforge3/envs/21cmfast/lib/python3.11/site-packages/py21cmfast/_utils.py:815: UserWarni

[1000/100000] done
[2000/100000] done
[3000/100000] done
[4000/100000] done
[5000/100000] done
[6000/100000] done
[7000/100000] done
[8000/100000] done
[9000/100000] done
[10000/100000] done
[11000/100000] done
[12000/100000] done
[13000/100000] done
[14000/100000] done
[15000/100000] done
[16000/100000] done
[17000/100000] done
[18000/100000] done
[19000/100000] done
[20000/100000] done
[21000/100000] done
[22000/100000] done
[23000/100000] done
[24000/100000] done
[25000/100000] done
[26000/100000] done
[27000/100000] done
[28000/100000] done
[29000/100000] done
[30000/100000] done
[31000/100000] done
[32000/100000] done
[33000/100000] done
[34000/100000] done
[35000/100000] done
[36000/100000] done
[37000/100000] done
[38000/100000] done
[39000/100000] done
[40000/100000] done
[41000/100000] done
[42000/100000] done
[43000/100000] done
[44000/100000] done
[45000/100000] done
[46000/100000] done
[47000/100000] done
[48000/100000] done
[49000/100000] done
[50000/100000] done
[51000/10

In [5]:
p21c.cache_tools.clear_cache(direc="_cache")

In [6]:
# Assumes these are already defined in your environment:
# build_21cm_fields(HII_DIM, BOX_LEN, redshift, RANDOM_SEED) -> (velocity_field_kms, density_field)
# lyalpha_forest_from_fields(velocity_field_kms, density_field, redshift, BOX_LEN, T0, gamma, y0, z0) -> dict with key "flux_F2"

# ----------------------------
# User controls
# ----------------------------
REDSHIFT = 2.5
HII_DIM  = 100
BOX_LEN  = 100.0

# Uniform parameter ranges (T0, gamma)
T0_MIN, T0_MAX       = 8000.0, 18000.0
GAMMA_MIN, GAMMA_MAX = 1.2, 1.7

# Nuisance parameter: tau scaling A = exp(logA)
# tau' = A * tau_true  ->  F' = exp(-A*tau_true) = F_true**A
LOGA_MIN, LOGA_MAX = -0.4, 0.4   # A in ~[0.67, 1.49]

# Noise controls (fixed SNR in FLUX space)
SNR_FIXED = 20.0
EPS_FLUX  = 1e-6  # keep flux away from exactly 0/1 (stable logs / transforms)
RNG_SEED  = 123

# Box realizations + sims per realization
N_REALIZATIONS = 20
N_SIMS_PER_REALIZATION = 1250
N_SIMS = N_REALIZATIONS * N_SIMS_PER_REALIZATION

OUTFILE = (
    f"lyalpha_test_z{REDSHIFT:.2f}_uniformT0gamma_logAUniform"
    f"Nsims{N_SIMS}_R{N_REALIZATIONS}_SNR{int(SNR_FIXED)}.npz"
)

# ----------------------------
# 0) Build fields ONCE for a dry run to get Npix
# ----------------------------
velocity_field_kms, density_field = build_21cm_fields(
    HII_DIM=HII_DIM,
    BOX_LEN=BOX_LEN,
    redshift=REDSHIFT,
    RANDOM_SEED=RNG_SEED + 0
)

# ----------------------------
# 1) RNG + deterministic dry run to get Npix
# ----------------------------
rng = np.random.default_rng(RNG_SEED)

T0_probe    = 0.5 * (T0_MIN + T0_MAX)
gamma_probe = 0.5 * (GAMMA_MIN + GAMMA_MAX)

test = lyalpha_forest_from_fields(
    velocity_field_kms=velocity_field_kms,
    density_field=density_field,
    redshift=REDSHIFT,
    BOX_LEN=BOX_LEN,
    T0=float(T0_probe),
    gamma=float(gamma_probe),
    y0=0,
    z0=0
)

F2_test = np.asarray(test["flux_F2"], dtype=np.float32)
Npix = int(F2_test.shape[0])

# Allocate dataset arrays
X     = np.empty((N_SIMS, Npix), dtype=np.float32)     # noisy transmitted flux after nuisance tau scaling
theta = np.empty((N_SIMS, 3), dtype=np.float32)        # [:,0]=T0, [:,1]=gamma, [:,2]=logA
yz    = np.empty((N_SIMS, 2), dtype=np.int32)          # [:,0]=y0, [:,1]=z0

# Precompute flux-noise sigma for fixed SNR
sigma = 1.0 / float(SNR_FIXED)

# ----------------------------
# 2) Generate dataset
# ----------------------------
idx = 0

for r in range(N_REALIZATIONS):

    # Build fields ONCE per realization
    velocity_field_kms, density_field = build_21cm_fields(
        HII_DIM=HII_DIM,
        BOX_LEN=BOX_LEN,
        redshift=REDSHIFT,
        RANDOM_SEED=RNG_SEED + r
    )

    for n in range(N_SIMS_PER_REALIZATION):

        # --- Uniform draws of physical parameters ---
        T0    = float(rng.uniform(T0_MIN, T0_MAX))
        gamma = float(rng.uniform(GAMMA_MIN, GAMMA_MAX))

        # --- Uniform draw of nuisance parameter logA (A = exp(logA)) ---
        logA = float(rng.uniform(LOGA_MIN, LOGA_MAX))
        A = float(np.exp(logA))

        # --- Random sightline position ---
        y0 = int(rng.integers(0, HII_DIM, dtype=np.int32))
        z0 = int(rng.integers(0, HII_DIM, dtype=np.int32))

        out = lyalpha_forest_from_fields(
            velocity_field_kms=velocity_field_kms,
            density_field=density_field,
            redshift=REDSHIFT,
            BOX_LEN=BOX_LEN,
            T0=T0,
            gamma=gamma,
            y0=y0,
            z0=z0
        )

        # Noiseless transmitted flux from your forward model
        F = np.asarray(out["flux_F2"], dtype=np.float32)

        # --- Apply nuisance in tau-space: tau' = A * tau_true ---
        # Safety clipping before log
        F = np.clip(F, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)
        tau = (-np.log(F)).astype(np.float32)
        F_scaled = np.exp(-A * tau).astype(np.float32)

        # --- Add Gaussian noise in FLUX space ---
        noise = rng.normal(loc=0.0, scale=sigma, size=Npix).astype(np.float32)
        F_noisy = np.clip(F_scaled + noise, EPS_FLUX, 1.0 - EPS_FLUX).astype(np.float32)

        # Store
        X[idx, :] = F_noisy
        theta[idx, 0] = T0
        theta[idx, 1] = gamma
        theta[idx, 2] = logA
        yz[idx, 0] = y0
        yz[idx, 1] = z0

        idx += 1
        if (idx % 1000 == 0) or (idx == N_SIMS):
            print(f"[{idx}/{N_SIMS}] done")

assert idx == N_SIMS, f"Filled idx={idx} rows, expected N_SIMS={N_SIMS}"

# ----------------------------
# 3) Save dataset
# ----------------------------
meta = {
    "redshift": float(REDSHIFT),
    "HII_DIM": int(HII_DIM),
    "BOX_LEN": float(BOX_LEN),

    "theta_sampling": "uniform_linear",
    "T0_range_K": [float(T0_MIN), float(T0_MAX)],
    "gamma_range": [float(GAMMA_MIN), float(GAMMA_MAX)],
    "logA_range": [float(LOGA_MIN), float(LOGA_MAX)],
    "theta_columns": ["T0_K", "gamma", "logA"],

    "nuisance_definition": "A = exp(logA); tau_scaled = A * tau_true; F_scaled = exp(-tau_scaled) = F_true**A",

    "N_realizations": int(N_REALIZATIONS),
    "N_sims_per_realization": int(N_SIMS_PER_REALIZATION),
    "N_sims": int(N_SIMS),
    "Npix": int(Npix),
    "rng_seed": int(RNG_SEED),

    "noise_model": "flux_additive_gaussian_after_tau_scaling",
    "snr_distribution": "fixed",
    "snr_fixed": float(SNR_FIXED),
    "sigma_definition": "sigma_flux = 1/SNR_FIXED (constant per pixel within a spectrum)",
    "flux_clipping": f"[{EPS_FLUX}, {1.0 - EPS_FLUX}]",
    "X_definition": "X is noisy transmitted flux after tau-scaling nuisance: F_noisy = clip(exp(-exp(logA)*(-ln(F_true))) + noise)",
    "summary": "X is noisy flux after tau-scaling; theta[:,0]=T0[K], theta[:,1]=gamma, theta[:,2]=logA; yz[:,0]=y0, yz[:,1]=z0.",
}

np.savez_compressed(
    OUTFILE,
    X=X,
    theta=theta,
    yz=yz,
    meta=np.array([meta], dtype=object)
)

print(f"Saved: {OUTFILE}")
print("X shape:", X.shape, "theta shape:", theta.shape, "yz shape:", yz.shape)
print("theta columns:", meta["theta_columns"])
print("theta ranges:",
      f"T0[{theta[:,0].min():.1f}, {theta[:,0].max():.1f}]",
      f"gamma[{theta[:,1].min():.3f}, {theta[:,1].max():.3f}]",
      f"logA[{theta[:,2].min():.3f}, {theta[:,2].max():.3f}]")


[1000/25000] done
[2000/25000] done
[3000/25000] done
[4000/25000] done
[5000/25000] done
[6000/25000] done
[7000/25000] done
[8000/25000] done
[9000/25000] done
[10000/25000] done
[11000/25000] done
[12000/25000] done
[13000/25000] done
[14000/25000] done
[15000/25000] done
[16000/25000] done
[17000/25000] done
[18000/25000] done
[19000/25000] done
[20000/25000] done
[21000/25000] done
[22000/25000] done
[23000/25000] done
[24000/25000] done
[25000/25000] done
Saved: lyalpha_test_z2.50_uniformT0gamma_logAUniformNsims25000_R20_SNR20.npz
X shape: (25000, 100) theta shape: (25000, 3) yz shape: (25000, 2)
theta columns: ['T0_K', 'gamma', 'logA']
theta ranges: T0[8000.1, 17999.1] gamma[1.200, 1.700] logA[-0.400, 0.400]
